In [1]:
import veloagent
import anndata
import torch
import scvelo as scv
import scanpy as sc

Global seed set to 0


## Load AnnData

In [2]:
adata = anndata.read_h5ad('/Users/brentyoon/Documents/COMP402/code/data/va_stereo+vae.h5ad')

## Preprocessing dataset

In [ ]:
veloagent.preprocess(adata)

## VAE Embedding

In [ ]:
vae, optimizer, loss_fn = veloagent.get_vae(adata, z_dim=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae.to(device)
global_nb_indices = adata.uns['neighbors']['indices']

In [ ]:
best_model = veloagent.train_vae(adata, vae, optimizer, loss_fn, global_nb_indices, patience=25, num_epochs=500, batch=0.25)

In [ ]:
vae.load_state_dict(best_model)
veloagent.get_embedding(adata, vae, device)

In [3]:
paths = veloagent.load_protein_paths(species="mouse", base="/Users/brentyoon/Documents/COMP402/code/veloagent/data/conn_mat")
protein_names_path = paths[0]
protein_alias_path = paths[1]
gene_conn_path = paths[2]

In [4]:
conn_mat = veloagent.create_con_mat(adata, adata.n_vars, protein_names_path, protein_alias_path, gene_conn_path, varname='index')

In [5]:
genenet = veloagent.GeneNet(in_dim=adata.obsm['cell_embed'][0].shape[0], gene_dim=adata.n_vars, conn=conn_mat)
optimizer = torch.optim.Adam(genenet.parameters(), lr=0.0001, weight_decay=0.00001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
genenet.to(device)

GeneNet(
  (dropout): Dropout(p=0.2, inplace=False)
  (lyr1): Linear(in_features=7, out_features=536, bias=True)
  (bn1): BatchNorm1d(536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lyr2): Linear(in_features=536, out_features=1072, bias=True)
  (bn2): BatchNorm1d(1072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lyr3): CustomizedLinear(input_features=1072, output_features=1072, bias=False)
  (bn3): BatchNorm1d(1072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lyr4): Linear(in_features=1072, out_features=2144, bias=True)
  (bn4): BatchNorm1d(2144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lyr5): Linear(in_features=2144, out_features=3216, bias=True)
)

In [ ]:
veloagent.train_gg(num_epochs=100, data=adata, embed_basis='cell_embed', genenet=genenet, device=device, optimizer=optimizer, patience=10, num_nbrs=30, dt=0.3, batch=0.25)

 10%|██▊                         | 10/100 [00:37<05:31,  3.68s/it, loss=1.28e+5]

In [ ]:
veloagent.train_nbr(num_epochs=25, data=adata, embed_basis='cell_embed', genenet=genenet, device=device, optimizer=optimizer, num_nbrs=15, dt=0.3, batch=0.25)

In [ ]:
bdata = anndata.read_h5ad('/Users/brentyoon/Documents/COMP402/code/data/va_stereo+vae.h5ad')

In [ ]:
adata.obsm["X_umap"] = bdata.obsm["X_umap"]

In [ ]:
scv.tl.velocity_graph(adata, vkey='velocity')
scv.pl.velocity_embedding_stream(adata,
                                legend_loc="right margin", 
                                vkey='velocity', 
                                basis='umap', 
                                color=['scc_anno'],
                                dpi=150)

In [ ]:
adata.obs['x_loc'] = adata.obsm['spatial'].T[0]/5
adata.obs['y_loc'] = adata.obsm['spatial'].T[1]/5
adata.obs['x_loc'] = adata.obs.x_loc.astype(int)
adata.obs['y_loc'] = adata.obs.y_loc.astype(int)
adata.obs['x_loc'] = adata.obs['x_loc'] - min(adata.obs['x_loc'])
adata.obs['y_loc'] = adata.obs['y_loc'] - min(adata.obs['y_loc'])

In [ ]:
abm = veloagent.CellModel(adata, steps=100, freedom=2, nbr_radius=40, sig_ratio=0.7)

In [ ]:
abm.step()

In [ ]:
scv.tl.velocity_graph(adata, vkey='velocity')
scv.pl.velocity_embedding_stream(adata,
                                legend_loc="right margin", 
                                vkey='velocity', 
                                basis='spatial', 
                                color=['scc_anno'],
                                palette="tab20",
                                dpi=150)